In [2]:
import os
import sys
import json
import logging
from dotenv import load_dotenv 
load_dotenv()
sys.path.append(os.getenv("PROJECT_PATH"))
from src.utils.azure_client import AzureClient, get_answer

from src.Evaluators.gpt_evaluator import GPTEvaluator

from src.Obfuscators.few_prompt_obfuscator import FewPromptsObfuscator
from src.Obfuscators.random_text import RandomText
from src.Obfuscators.totally_random_emoji import TotallyRandomEmoji


In [51]:
disable_httpx_log = logging.getLogger("httpx")
disable_httpx_log.setLevel(logging.WARNING)
logging.basicConfig(filename="random_log", level=logging.INFO)
logger = logging.getLogger(__name__)

In [52]:
two_prompt_1_path = os.path.join(os.getenv("PROJECT_PATH"),"src","Obfuscators","prompts","two_querries", "first_querry.txt")
two_prompt_2_path = os.path.join(os.getenv("PROJECT_PATH"),"src","Obfuscators","prompts","two_querries", "second_querry_random_emoji.txt")

azure_llm_wrapper_factory = lambda :AzureClient("azure_client",  "../", "gpt-4o-2024-05-13", 0.0)
with open(two_prompt_1_path, 'r', encoding='utf-8') as file:
    two_prompt_1 = file.read()
with open(two_prompt_2_path, 'r', encoding='utf-8') as file:
    two_prompt_2 = file.read()


cpprefix = "Do not explain the emojis in your answer.\n"
smart_random_emoji = lambda : FewPromptsObfuscator([two_prompt_1,two_prompt_2], azure_llm_wrapper_factory, logger, cpprefix)
totally_random_emoji = lambda : TotallyRandomEmoji()
random_text = lambda : RandomText(azure_llm_wrapper_factory)

gpt_evaluator = GPTEvaluator(logger, "..")

GPT EVALUATOR INIT


In [53]:
data_to_use = "RESULTS_2024-08-07_10_46_47.771852gpt_metric_new_test.json"
inputfile_path = os.path.join(os.getenv("PROJECT_PATH"),"data",data_to_use)
with open(inputfile_path, 'r') as file:
    data = json.load(file)

In [54]:
original_prompt = "How does the use of machine learning algorithms in predictive analytics enhance decision-making processes in businesses, and what are some common challenges faced during implementation? BE CONCISE"
original_answer = "Machine learning algorithms enhance decision-making in businesses by analyzing large datasets to identify patterns, predict outcomes, and provide actionable insights. This leads to more informed, data-driven decisions, improved efficiency, and competitive advantages.\n\nCommon challenges during implementation include:\n\n1. **Data Quality**: Incomplete or inaccurate data can lead to poor model performance.\n2. **Complexity**: Developing and tuning models requires specialized skills and knowledge.\n3. **Integration**: Incorporating machine learning systems into existing workflows and IT infrastructure can be difficult.\n4. **Scalability**: Ensuring models perform well as data volume grows.\n5. **Bias**: Models can perpetuate or amplify existing biases in data.\n6. **Cost**: High initial investment in technology and talent.\n\nAddressing these challenges requires careful planning, ongoing monitoring, and a commitment to data governance and ethical practices."

In [58]:
obfuscated_prompt = totally_random_emoji().obfuscate(original_prompt)
print(obfuscated_prompt)

How does the use of machine learning algorithms in predictive analytics enhance decision-making processes in businesses, and what are some common challenges faced during implementation? BE CONCISE


In [56]:
print(get_answer("What is the capital of France?",model = "gpt-35-turbo"))

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

In [ ]:
results_arr = []
for i in range (1):
    obfuscated_prompt = totally_random_emoji().obfuscate(original_prompt)
    prompt_evaluation = gpt_evaluator.evaluate_prompt(original_prompt, obfuscated_prompt)
    answer = AzureClient.get_answer(original_prompt)
    answer_evaluation = gpt_evaluator.evaluate_prompt(original_answer, answer)
    print("""
obfuscated_prompt: {obfuscated_prompt}
prompt_evaluation: {prompt_evaluation}
answer: {answer}
answer_evaluation: {answer_evaluation}
          """)
    results_arr.append(prompt_evaluation)

AuthenticationError: Error code: 401 - {'statusCode': 401, 'message': 'Access denied due to invalid subscription key. Make sure to provide a valid key for an active subscription.'}

In [25]:
import unicodedata
import json
import os

text1 =  "**Advantages of NoSQL for Large-Scale, Unstructured Data:**\n\n1. **Scalability:** NoSQL databases are designed to scale out by adding more servers, making them ideal for handling large volumes of data.\n2. **Flexibility:** They can store unstructured, semi-structured, and structured data without requiring a fixed schema, allowing for more agile development.\n3. **Performance:** Optimized for read and write operations, NoSQL databases can offer faster performance for certain types of queries and workloads.\n4. **Distributed Architecture:** Many NoSQL databases are built to be distributed, providing high availability and fault tolerance.\n\n**Disadvantages of NoSQL for Large-Scale, Unstructured Data:**\n\n1. **Consistency:** NoSQL databases often sacrifice consistency (in favor of availability and partition tolerance) due to the CAP theorem, which can complicate data integrity.\n2. **Complexity:** Managing and querying unstructured data can be more complex and may require specialized knowledge.\n3. **Maturity:** NoSQL databases are generally newer and may lack some of the robust tooling and community support available for traditional relational databases.\n4. **Limited ACID Transactions:** Many NoSQL databases do not fully support ACID transactions, which can be a drawback for applications requiring strong transactional integrity."
text2 =  """Advantages of NoSQL database:
- 🌐⚡
- 💰⬇️
- 🛠️🔄

Disadvantages of NoSQL database:
- 📶❌
- 🔒❓
- 🕒⬆️
"""
print(text1)
print(text2)

**Advantages of NoSQL for Large-Scale, Unstructured Data:**

1. **Scalability:** NoSQL databases are designed to scale out by adding more servers, making them ideal for handling large volumes of data.
2. **Flexibility:** They can store unstructured, semi-structured, and structured data without requiring a fixed schema, allowing for more agile development.
3. **Performance:** Optimized for read and write operations, NoSQL databases can offer faster performance for certain types of queries and workloads.
4. **Distributed Architecture:** Many NoSQL databases are built to be distributed, providing high availability and fault tolerance.

**Disadvantages of NoSQL for Large-Scale, Unstructured Data:**

1. **Consistency:** NoSQL databases often sacrifice consistency (in favor of availability and partition tolerance) due to the CAP theorem, which can complicate data integrity.
2. **Complexity:** Managing and querying unstructured data can be more complex and may require specialized knowledge.
3

In [30]:
answer_prompt = """give a number between 0.0 to 1.0 where 0.0 is not similar at all and 1.0 signifies that the texts are highly similar for a human to understand. The two following texts:
Text1: {text1}
Text2: {text2}
Please provide the similarity score along with a brief explanation of the rationale behind the score. Print $ANSWER: <number>"""


def evaluate_answer(answer_prompt, obfuscated_answer, original_answer):
        query = answer_prompt.format(text1=obfuscated_answer, text2=original_answer)
        answer = get_answer(query)
        print(f"""evaluate_answer
Prompt: {query}
Answer: {answer}""")
        
print(evaluate_answer(answer_prompt, text2, text1))


evaluate_answer
Prompt: give a number between 0.0 to 1.0 where 0.0 is not similar at all and 1.0 signifies that the texts are highly similar for a human to understand. The two following texts:
Text1: Advantages of NoSQL database:
- 🌐⚡
- 💰⬇️
- 🛠️🔄

Disadvantages of NoSQL database:
- 📶❌
- 🔒❓
- 🕒⬆️

Text2: **Advantages of NoSQL for Large-Scale, Unstructured Data:**

1. **Scalability:** NoSQL databases are designed to scale out by adding more servers, making them ideal for handling large volumes of data.
2. **Flexibility:** They can store unstructured, semi-structured, and structured data without requiring a fixed schema, allowing for more agile development.
3. **Performance:** Optimized for read and write operations, NoSQL databases can offer faster performance for certain types of queries and workloads.
4. **Distributed Architecture:** Many NoSQL databases are built to be distributed, providing high availability and fault tolerance.

**Disadvantages of NoSQL for Large-Scale, Unstructured

In [20]:
inputfile_path = os.path.join(os.getenv("PROJECT_PATH"),"data","RESULTS_gpt_metric_new_test2024-08-08_18_08_45.489784 - Smart Random Emoji - New Prefix.json")
with open(inputfile_path, 'r') as file:
        data = json.load(file)
for obf in data:
    if obf[0] == "RandomText":
        for dict_elem in obf[1]:
            if "Can you explain the concept of containerization and how Docker helps in creating, deploying, and running applications within containers? BE CONCISE" in dict_elem["original_prompt"]:
                print(dict_elem["obfuscated_dictonary"])

{'containerization': '🐳📦', 'Docker': '🐋⚓', 'creating applications': '🛠️📱', 'deploying applications': '🚀📤', 'running applications': '🏃\u200d♂️💻', 'containers': '📦🚢'}


In [14]:
from src.Obfuscators.wrong_obfuscator import WrongObfuscator
factory = lambda : WrongObfuscator()

print(type(factory))

<class 'function'>
